# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.114382e+02     1.622129e+00
 * time: 0.07538104057312012
     1     1.092347e+01     8.555678e-01
 * time: 1.3986101150512695
     2    -1.141735e+01     1.020042e+00
 * time: 1.498924970626831
     3    -3.388589e+01     7.463936e-01
 * time: 1.6291470527648926
     4    -4.730988e+01     5.281953e-01
 * time: 1.7802090644836426
     5    -5.690486e+01     2.262305e-01
 * time: 1.8982000350952148
     6    -5.976492e+01     1.593242e-01
 * time: 1.9873430728912354
     7    -6.091225e+01     8.544111e-02
 * time: 2.0774829387664795
     8    -6.140782e+01     5.239465e-02
 * time: 2.182513952255249
     9    -6.168674e+01     3.339579e-02
 * time: 2.2729270458221436
    10    -6.184799e+01     2.725793e-02
 * time: 2.3635520935058594
    11    -6.199392e+01     2.033551e-02
 * time: 2.4535181522369385
    12    -6.208906e+01     1.851785e-02
 * time: 2.5445921421051025
    13    -6.213798e+01     1.616094e-02
 * time: 2.64530110359

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557664
    AtomicNonlocal      14.8522636
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336819

    total               -62.261666462501
